In [ ]:
import numpy as np
import pandas as pd
# from sqlalchemy import create_engine

In [ ]:
# engine = create_engine('sqlite:///data/test.db')
# DF = pd.read_sql("Select * from dm_POS_DB where BASE_DT='20190211'", engine)
# df = DF[DF['TAG']=='Master'].iloc[3:40]
# pd.to_pickle(df, 'data.p')

In [ ]:
df = pd.read_pickle('data.p')

In [ ]:
df.iloc[0]

In [ ]:
cols = ['TICKER', 'CTRY', 'GICS_CD', 'MktVal']
df0 = df[cols]

In [ ]:
df1 = df0.set_index(['CTRY', 'GICS_CD']).sort_index()

In [ ]:
# def looper(item, p, df, idx=[]):    
#     if p:
#         a = dict()
#         a['name'] = item
#         a['children'] = list()
#         return a, idx + [item]
#     else:
#         rname = {'TICKER': 'name', 'MktVal': 'value'}
#         a = dict()
#         a['name'] = item
#         a['children'] = df.loc[tuple(idx + [item])] \
#             .rename(columns=rname) \
#             .to_dict(orient='records')
#         return a, None

In [ ]:
# lst, p = idx_spitter(df1.loc['JP'].index)
# idx=['JP']

In [ ]:
def idx_spitter(idx):
    if isinstance(idx, pd.core.index.MultiIndex):
        return set(idx.levels[0].tolist()), True
    else:
        return set(idx.tolist()), False

In [ ]:
def looper(df, p, item=None, idx=[]):
    if item is None:
        a = dict()
        a['name'] = 'port'
        a['children'] = list()
        return a, []
    else:
        if p:
            a = dict()
            a['name'] = item
            a['children'] = list()
            return a, idx + [item]
        else:
            rname = {'TICKER': 'name', 'MktVal': 'value'}
            a = dict()
            a['name'] = item
            a['children'] = df.loc[tuple(idx + [item])] \
                .rename(columns=rname) \
                .to_dict(orient='records')
            return a, None

In [ ]:
# a = dict()
# a['name'] = 'port'
# a['children'] = list()
# for item in lst:
#     a['children'].append(looper(item, p, idx=idx)[0])

In [ ]:
a = dict()
a['name'] = 'port'
a['children'] = list()
lst, p = idx_spitter(df1.index)
idx=[]
for item in lst:
    # 1st level
    a_, idx_ = looper(df1, p, item, idx=idx)
    lst_, p_ = idx_spitter(df1.loc[a_['name']].index)
    if not p:
        a['children'].append(a_)
    else:
        for item_ in lst_:
            # 2nd level
            a__, idx__ = looper(df1, p_, item_, idx=idx_)
            if not p_:
                a_['children'].append(a__)
    a['children'].append(a_)

In [ ]:
def main(df, p=True, item=None, idx=[]):
    a, idx = looper(df, p, item, idx=idx)
    if item is None:
        lst, p = idx_spitter(df.index)
    else:
        if not p:
            lst, p = None, False
        else:
            lst, p = idx_spitter(df.loc[a['name']].index)
    return a, idx, lst, p

In [ ]:
%qtconsole

In [ ]:
a, idx, lst, p = main(df1, p=True, item=None, idx=[])
for item in lst:
    a_, idx_, lst_, p_ = main(df1, p, item, idx=idx)
    if not p:
        a['children'].append(a_)
    else:
        for item_ in lst_:
            a__, idx__, lst__, p__ = main(df1, p_, item_, idx=idx_)
            if not p_:
                a_['children'].append(a__)
    a['children'].append(a_)

In [ ]:
import json

In [ ]:
json.dumps(a)